In [1]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leonardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-unicos-lematizados.csv', sep = '|', encoding = 'utf-8')
df.head()

,acordao,areas,texto_lematizado
0,297/2016-P,Responsabilidade,tribunal de conta de o união tc XXNUMERACAO\nX...
1,366/2016-P,Finanças Públicas,tribunal de conta de o união tc XXNUMERACAO\nX...
2,944/2016-P,Responsabilidade,tribunal de conta de o união tc XXNUMERACAO\nX...
3,30/2016-P,Direito Processual,tribunal de conta de o união tc XXNUMERACAO\nX...
4,55/2016-P,Pessoal,; - ; ; wania lucia pasquarelli de o nasciment...


In [3]:
df.shape

(9739, 3)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 

textos = df['texto_lematizado'].values
vetorizador = TfidfVectorizer(stop_words = stopwords.words('portuguese'))  #token_pattern=r'\S+', 
vetorizador.fit(textos)
print(len(vetorizador.vocabulary_), 'termos distintos')

135748 termos distintos


In [6]:
matriz = vetorizador.transform(textos)
matriz[0]

<1x135748 sparse matrix of type '<class 'numpy.float64'>'
	with 607 stored elements in Compressed Sparse Row format>

In [7]:
def extrair_tfidf(documento):
    indices = []
    acordao = []
    tf_idf = []
    for termo in textos[documento].split():
        if termo in vetorizador.vocabulary_:
            indice = vetorizador.vocabulary_[termo]
            indices.append(indice)
            acordao.append(termo)
            tf_idf.append(matriz[documento,indice])
    return acordao, indices, tf_idf

In [9]:
acordao, indice, tf_idf = extrair_tfidf(9000)
for i in range(0,21):
    print(acordao[i], '\t', indice[i], '\t', tf_idf[i])

tribunal 	 124604 	 0.011890583954143085
conta 	 25317 	 0.007318785109663216
união 	 126354 	 0.002744262649844002
tc 	 121256 	 0.0019072084518832392
grupo 	 64704 	 0.0018293206083297054
classe 	 20875 	 0.0018293206083297054
ver 	 127861 	 0.010998484652063512
natureza 	 89245 	 0.0018302599257358791
representação 	 107538 	 0.023312792729245184
entidade 	 52259 	 0.004435813284773901
instituto 	 72093 	 0.037621658247554285
brasileiro 	 14163 	 0.02897854097053326
museu 	 88438 	 0.08719486406265776
ibram 	 67916 	 0.39288447426403994
representante 	 107516 	 0.0027456717706640073
secretaria 	 112846 	 0.008091574783643505
fiscalizar 	 58633 	 0.07943248017928838
tecnologia 	 121532 	 0.057379807757647644
informação 	 71288 	 0.0440336612132619
sefti 	 113053 	 0.1021591093072635
sumário 	 119438 	 0.000917858749298157


In [11]:
len(acordao), len(indice), len(tf_idf)

(6768, 6768, 6768)

In [13]:
def extrair_tamanho(documento):
    qtde = 0
    for termo in textos[documento].split():
        if termo in vetorizador.vocabulary_:
            qtde += 1
    return qtde
extrair_tamanho(9000)

6768

In [14]:
for doc in range(0, len(textos)):
    df.loc[doc, 'tamanho'] = extrair_tamanho(doc)
df['tamanho'].describe()

count      9739.000000
mean       4845.784988
std       10068.222161
min         283.000000
25%        1766.500000
50%        3078.000000
75%        5588.000000
max      604173.000000
Name: tamanho, dtype: float64

In [15]:
import numpy as np

np.percentile(df['tamanho'], 90), np.percentile(df['tamanho'], 95), np.percentile(df['tamanho'], 99)

(9848.000000000004, 13933.700000000003, 26882.500000000022)

In [16]:
def filtrar_texto(acordao, tf_idf, maximo):
    derivado = ''
    original = len(tf_idf)
    if maximo >= original:
        minimo = 0
    else:
        minimo = np.percentile(tf_idf,100-100*maximo/original)
    for indice, valor in enumerate(tf_idf):
        if valor > minimo:
            derivado += acordao[indice] + ' '
    return derivado[:-1]
filtrar_texto(acordao, tf_idf, 1000)

'museu ibram sefti contrato serviço fiscal gestor contrato serviço gestor contrato fiscal sefti peça contrato museu ibram contrato ser serviço peça ser serviço ser peça peça ibram avaliar serviço avaliar serviço serviço fiscal peça peça ibram ibram peça ibram peça avaliar sefti peça contrato fiscal maxwell contrato peça sefti peça gestor contrato lemgruber junior serviço contrato fiscal peça sefti peça sefti contrato maxwell lemgruber junior peça serviço serviço peça ser ser peça ser contrato peça ser peça serviço ser ser peça avaliar serviço ibram peça contrato fiscal maxwell gestor contrato lemgruber junior ser ti ibram avaliar serviço fiscal fiscal avaliar gestor contrato peça peça fiscal sefti peça peça ser maxwell peça fiscal contrato gestor contrato fiscal contrato fiscal avaliar slti peça maxwell ibram peça contrato ti peça contrato ibram gestor contrato fiscal contrato peça contrato peça ser avaliar ser peça gestor peça peça maxwell avaliar gestor contrato ser slti ser contrato

In [24]:
filtrar_texto(acordao, tf_idf, 8000)

'tribunal conta união tc grupo classe ver natureza representação entidade instituto brasileiro museu ibram representante secretaria fiscalizar tecnologia informação sefti sumário representação supor regularidade ocorrer execução contrato prestar serviço suporte tecnológico divergência fiscal técnico gestor contrato atesto fatura ausência plano fiscalizar análise qualidade atesto pagar serviço apenas gestor contrato manifestação prévio fiscal técnico conhecer procedência parcial determinação ciência entidade contratante relatório tratar representação formular secretaria fiscalizar tecnologia informação sefti motivar manifestação registrada ouvidoria peça respeito suposto regularidade ocorrer contrato firmar instituto brasileiro museu ibram empresa stefanini consultadoria assessoria informática cnpj aludir contrato ser resultante pregão eletrônico pe ter objeto dprestação serviço suporte tecnológico ambiente tecnologia informação comunicação tic contratante unidade compor estrutura organ

In [26]:
for doc in range(0, len(textos)):
    acordao, _ , tf_idf = extrair_tfidf(doc)
    df.loc[doc, 'filtrado_6000'] = filtrar_texto(acordao, tf_idf, 6000)
    df.loc[doc, 'filtrado_500']  = filtrar_texto(acordao, tf_idf, 500)
    if (doc + 1) % 1000 == 0:
        print('doc', doc+1, '...')

doc 1000 ...
doc 2000 ...
doc 3000 ...
doc 4000 ...
doc 5000 ...
doc 6000 ...
doc 7000 ...
doc 8000 ...
doc 9000 ...


In [27]:
df.head(10)

,acordao,areas,texto_lematizado,tamanho,filtrado_6000,filtrado_500
0,297/2016-P,Responsabilidade,tribunal de conta de o união tc XXNUMERACAO\nX...,1704.0,tribunal conta união tc grupo classe plenário ...,conta conta gerência instituto nacional seguro...
1,366/2016-P,Finanças Públicas,tribunal de conta de o união tc XXNUMERACAO\nX...,3330.0,tribunal conta união tc grupo classe ii plenár...,senado petróleo petróleo regime aduaneiro expo...
2,944/2016-P,Responsabilidade,tribunal de conta de o união tc XXNUMERACAO\nX...,842.0,tribunal conta união tc grupo classe plenário ...,tribunal conta plenário embargo declaração aco...
3,30/2016-P,Direito Processual,tribunal de conta de o união tc XXNUMERACAO\nX...,1379.0,tribunal conta união tc grupo classe plenário ...,embargo declaração inss recorrente marco antôn...
4,55/2016-P,Pessoal,; - ; ; wania lucia pasquarelli de o nasciment...,5887.0,wania lucia pasquarelli nascimentotcuwania luc...,senac senac senac empresa senac giselle araújo...
5,27/2016-P,Licitação,tribunal de conta de o união tc XXNUMERACAO\nX...,2025.0,tribunal conta união tc grupo classe plenário ...,apenso recurso natal nilton pascoal cpf walden...
6,38/2016-P,Pessoal,tribunal de conta de o união tc XXNUMERACAO\nX...,3292.0,tribunal conta união tc grupo classe plenário ...,aposentadoria sebastião aposentadoria gratific...
7,113/2016-P,Licitação,tribunal de conta de o união tc XXNUMERACAO\nX...,3425.0,tribunal conta união tc grupo classe plenário ...,dixtal biomédico pró life equipar eireli dixta...
8,119/2016-P,Licitação,tribunal de conta de o união tc XXNUMERACAO\nX...,5943.0,tribunal conta união tc grupo classe plenário ...,snake empresa pregão pregão trt vigilância emp...
9,412/2016-1,Pessoal,tribunal de conta de o união tc XXNUMERACAO\nX...,3849.0,tribunal conta união tc grupo classe tc nature...,aposentadoria enoe pimenta basso aposentadoria...


In [28]:
df.to_csv('../dados/acordaos-unicos-filtrados-500.csv', sep = '|', columns = ['acordao', 'areas', 'filtrado_500'], index=False)

In [29]:
df.to_csv('../dados/acordaos-unicos-filtrados-6000.csv', sep = '|', columns = ['acordao', 'areas', 'filtrado_6000'], index=False)